# Desenvolvimento do algoritmo de apadrinhamento

---

In [1]:
import sys
sys.path.append('../app')
from main import (
    filtro_colunas,
    extrair_valores_unicos_ordenados,
    gerar_matriz_interesse,
    algoritmo_apadrinhamento_lp,
    algoritmo_apadrinhamento_2024
)
import pandas as pd

Importante suposições sobre o algoritmo função:
1. Estrutura das Colunas:

    O dataframe possui múltiplas colunas, e a função ```filtro_colunas``` espera que você forneça uma lista de índices de colunas para filtrar as colunas desejadas.

2. Conteúdo das Células:

    As células do dataframe podem conter valores separados por vírgulas. A função ```extrair_valores_unicos_ordenados``` divide as células em valores individuais com base nas vírgulas.

3. Formato dos Dados:

    As células contêm strings que podem ser divididas em múltiplos valores. A função ```extrair_valores_unicos_ordenados``` conta a frequência desses valores e os organiza.

4. Colunas Relevantes:

    A função ```extrair_valores_unicos_ordenados``` ignora a primeira coluna (df.columns[1:num]), sugerindo que a primeira coluna pode ser um índice ou uma coluna não relevante para a extração de valores únicos.

5. Valores Únicos:

    A função ```extrair_valores_unicos_ordenados``` está interessada em valores que aparecem mais de uma vez, removendo valores únicos ou vazios da lista final.

## Carregamento de dados


In [2]:
# Arquivos de dados
arquivo_afilhados = "..\\dados\\afilhados.csv"
arquivo_padrinhos = "..\\dados\\padrinhos.csv"

# Carregar dados
df_afilhados = pd.read_csv(arquivo_afilhados)
df_padrinhos = pd.read_csv(arquivo_padrinhos)


In [3]:
print("Afilhados:")
display(df_afilhados.head())

print("\nPadrinhos:")
display(df_padrinhos.head())

Afilhados:


,nome,interesse_1,interesse_2,interesse_3
0,Ana,"yoga, pilates, meditação, dança","culinária, gastronomia, enologia, mixologia","astronomia, astrofísica, cosmologia, espaço"
1,Julia,"futebol, basquete, vôlei, natação","culinária, gastronomia, enologia, mixologia","viagem, turismo, aventura, mochilão"
2,Carla,"futebol, basquete, vôlei, natação","cinema, teatro, música, dança","jardinagem, paisagismo, horticultura, botânica"
3,Mariana,"yoga, pilates, meditação, dança","leitura, escrita, poesia, literatura","viagem, turismo, aventura, mochilão"



Padrinhos:


,nome,interesse_1,interesse_2,interesse_3,quantidade_afilhados
0,João,"yoga, pilates, meditação, dança","futebol, basquete, vôlei, natação","yoga, pilates, meditação, dança",3
1,Maria,"culinária, gastronomia, enologia, mixologia","culinária, gastronomia, enologia, mixologia","leitura, escrita, poesia, literatura",7
2,José,"astronomia, astrofísica, cosmologia, espaço","viagem, turismo, aventura, mochilão","viagem, turismo, aventura, mochilão",6
3,Pedro,"futebol, basquete, vôlei, natação","leitura, escrita, poesia, literatura","astronomia, astrofísica, cosmologia, espaço",1


## Pré-processamento dos Dados


---

In [4]:
#Filtra as colunas necessárias para o algoritmo de apadrinhamento
# Filtrar colunas
indice_afilhados = [0,1,2,3]
indice_padrinhos = [0,1,2,3,4]

# Filtrar colunas
df_afilhados = filtro_colunas(df_afilhados, indice_afilhados)
df_padrinhos = filtro_colunas(df_padrinhos, indice_padrinhos)

In [5]:
# Transformando todas as células em string
df_afilhados = df_afilhados.map(str)
df_padrinhos.iloc[:, :-1] = df_padrinhos.iloc[:, :-1].map(str)

In [6]:
print("Afilhados após pré-processamento:")
display(df_afilhados.head())

print("\nPadrinhos após pré-processamento:")
display(df_padrinhos.head())

Afilhados após pré-processamento:


,nome,interesse_1,interesse_2,interesse_3
0,Ana,"yoga, pilates, meditação, dança","culinária, gastronomia, enologia, mixologia","astronomia, astrofísica, cosmologia, espaço"
1,Julia,"futebol, basquete, vôlei, natação","culinária, gastronomia, enologia, mixologia","viagem, turismo, aventura, mochilão"
2,Carla,"futebol, basquete, vôlei, natação","cinema, teatro, música, dança","jardinagem, paisagismo, horticultura, botânica"
3,Mariana,"yoga, pilates, meditação, dança","leitura, escrita, poesia, literatura","viagem, turismo, aventura, mochilão"



Padrinhos após pré-processamento:


,nome,interesse_1,interesse_2,interesse_3,quantidade_afilhados
0,João,"yoga, pilates, meditação, dança","futebol, basquete, vôlei, natação","yoga, pilates, meditação, dança",3
1,Maria,"culinária, gastronomia, enologia, mixologia","culinária, gastronomia, enologia, mixologia","leitura, escrita, poesia, literatura",7
2,José,"astronomia, astrofísica, cosmologia, espaço","viagem, turismo, aventura, mochilão","viagem, turismo, aventura, mochilão",6
3,Pedro,"futebol, basquete, vôlei, natação","leitura, escrita, poesia, literatura","astronomia, astrofísica, cosmologia, espaço",1


# Realização do Match Batch


---


In [7]:
# Extrair valores únicos e ordenados e o indice do inicio de cada coluna
colunas_afilhados, indice_inicio_colunas_afilhados = extrair_valores_unicos_ordenados(df_afilhados)
colunas_padrinhos, indice_inicio_colunas_padrinhos = extrair_valores_unicos_ordenados(df_padrinhos, -1)

# Cria um DataFrame vazio com as colunas extraidas
df_padrinhos_dummy = pd.DataFrame(columns=colunas_padrinhos + ['Numero de afilhados'])
df_afilhados_dummy = pd.DataFrame(columns=colunas_afilhados)

In [8]:
# Gera um dataframe com as informações de interesse
df_padrinhos_dummy = gerar_matriz_interesse(df_padrinhos, df_padrinhos_dummy, indice_inicio_colunas_padrinhos, 1)
df_afilhados_dummy = gerar_matriz_interesse(df_afilhados, df_afilhados_dummy, indice_inicio_colunas_afilhados,0)

In [9]:
# Aplica o algoritmo de apadrinhamento com programação linear
df_final_lp = algoritmo_apadrinhamento_lp(df_afilhados_dummy, df_padrinhos_dummy)
df_final_lp

,Nome afilhado,Compatibilidade,Nome padrinho
0,Ana,75.0,João
1,Julia,50.0,Maria
2,Carla,100.0,Pedro
3,Mariana,75.0,João


In [10]:
# Comparação simples com o algoritmo de apadrinhamento de 2024
df_final_2024 = algoritmo_apadrinhamento_2024(df_afilhados_dummy, df_padrinhos_dummy)
df_final_2024

,Nome bixo,Compatibilidade,Nome padrinho,Distância
0,Ana,75.0,Maria,4
1,Julia,50.0,Pedro,8
2,Carla,50.0,José,8
3,Mariana,75.0,José,4


In [11]:
'''df_final_lp.to_csv('resultado.csv', index=False)'''

"df_final_lp.to_csv('resultado.csv', index=False)"